## 110-2 Operations Research Case 2

Algorithm 3: based on Proc time/LST Ratio based on Algo 2
庭瑜的想法


## Loading data

In [2]:
import pandas as pd 
import numpy as np

In [3]:
# !git clone https://github.com/DanielStutzbach/heapdict

In [4]:
# see https://github.com/DanielStutzbach/heapdict
# or directly !pip install HeapDict

In [5]:
datadir = './data'
instances = []
for i in range(5):
    name = f'instance_{i+1}.csv'
    fullpath = datadir+'/'+name
    instances.append(pd.read_csv(fullpath))

## Data Structures

In [6]:
# job structure 
class Job:
    '''structure for 1 job '''
    def __init__(self, row):
        '''input := df.iloc[idx, :]'''
        self.id = row['Job ID']
        self.due = row['Due Time']
        self.next_op = 0 # True as complete, False as not yet processed
        self.stage_pt = [row['Stage-1 Processing Time'], row['Stage-2 Processing Time']]
        mfor1 = list(map(int, row['Stage-1 Machines'].split(',')))
        if row['Stage-2 Machines'] is not np.nan:
            mfor2 = list(map(int, row['Stage-2 Machines'].split(',')))
        else: mfor2 = [] 
        self.stage_mach = [mfor1, mfor2]
        self.assign_mach = [None for _ in range(2)]
        self.start_time = [-1 for _ in range(2)]
        self.end_time = [-1 for _ in range(2)]
    
    def __repr__(self):
        return f'\
          * Job id: {self.id}\n\
          * Due time:{self.due}\n\
          stage 1: {self.assign_mach[0]}\n\
                   {self.stage_pt[0]}, {self.stage_mach[0]}\n\
          stage 2: {self.assign_mach[1]}\n\
                   {self.stage_pt[1]}, {self.stage_mach[1]}'
    __str__ = __repr__
        

In [7]:
class Jobs:
    '''structure for multiple jobs' management'''
    def __init__(self, n):
        self.completion_times = np.zeros(n)
        self.tardiness = np.zeros(n)
        self.is_completed = np.full(n, False)
        self.residual_times = np.zeros(n)
        self.jobs = []
        
    def get_RRDD(self):
        if getattr(self, 'RRDD', None) is None:
            self.RRDD = self.due_dates - np.min(self.due_dates)
        return self.RRDD # static

    def get_LST(self):
        '''latest start time'''
        self.LST = self.due_dates - self.residual_times
        return self.LST
    
    def add_jobs(self, data):
        self.due_dates = data['Due Time'].to_numpy()
        for i in range(len(data)):
            row = data.iloc[i, :]
            jobi = Job(row)
            self.residual_times[i] = sum(jobi.stage_pt)
            self.jobs.append(jobi)
            
    
    def assign(self, job_name, mach, st):
        '''job_name = (2, 0) means job 3 and op 1
        note that job and op is 0-indexed as well as machines
        op
        '''
        
        i = 0 
        jobidx, op = job_name 
        job = self.jobs[jobidx]
        J.completion_times[jobidx] = st + job.stage_pt[op]
        J.residual_times[jobidx] -= job.stage_pt[op]
        job.assign_mach[op] = mach
        job.start_time[op] = st
        job.end_time[op] = J.completion_times[jobidx]
        job.next_op = op+1
        if op == 1:
            self.is_completed[jobidx] = True

In [8]:
class Machines:
    def __init__(self, df):
        '''pass the stage1, stage2 machine lists'''
        mfor1 = df['Stage-1 Machines'].values.tolist()
        mfor2 = df['Stage-2 Machines'].values.tolist()
        mfor1 = [list(map(int, x.split(','))) for x in mfor1]
        mfor2 = [list(map(int, x.split(','))) for x in mfor2 if x is not np.nan]
        mfor1 = [item for sublist in mfor1 for item in sublist]
        mfor2 = [item for sublist in mfor2 for item in sublist]
    
        self.number = max(max(mfor1), max(mfor2))
        self.versatile = [mfor1.count(i+1) + mfor2.count(i+1) for i in range(self.number)]
        self.schedule = [[] for _ in range(self.number)]
        self.span = [[] for _ in range(self.number)]
        self.fintime = [0 for _ in range(self.number)]
        
    def _schedule(self, mach, job_name, st, proc_time):
        '''mach is 0-indexed'''
        display_name = tuple([x+1 for x in job_name])
        self.schedule[mach].append(f'{display_name} {proc_time:.2f}') 
        self.span[mach].append(proc_time)
        self.fintime[mach] = st + proc_time
    
    def add_idle(self, mach, idle_time):
        self.schedule[mach].append(f'idle {idle_time:.2f}') 
        self.span[mach].append(idle_time)
        self.fintime[mach] += idle_time
        

## Which instance to test?

In [9]:
INSTANCEIDX = 4
df = instances[INSTANCEIDX]

## Preprocessing 
Usage:
1. Read the dataframe into `Machines()` as `M`.
2. Init by giving the length of jobs to `Jobs()` as `J`.
    Initialize it by calling `add_jobs()`.
3. Call our heuristic algorithm. 
4. Get the result from `M, J`. No need to return them. 

In [10]:
M = Machines(df)
J = Jobs(len(df))
J.add_jobs(df)
# print(J.residual_times)
# print(J.get_LST())

## Algorithm

#### Warnings:

1. Re-run the code from **preprcoessing section** otherwise the data stuctures will keep accumulating repetitive datas. 

2. All the indexing is 0-indexed for coding convenience, but when storing back to `M.schedule` for displaying purpose, it is changed into 1-indexed. 

# TO REVISE 5/4 11:54這裡還沒改可以先別看lolll
#### Steps:

1. ```LST = self.due_dates - self.residual_times ```

    `self.due_dates[j]` is the deadline for job ${j}$.
    `residual_times[j]` is the remaining processing times for job ${j}$.
    `LST[j]` is very similar to the idea of slackness/remaining time for you to procrastinate. 
    The less is LST, the sooner this job is to be scheduled (otherwise it's too late for it to catch up the deadline).
    


2. Jobs are put into a priority queue ${Q}$, which are ordered by their associated **LST**.


3. ```Extract_min()``` from ${Q}$ as `curr_job`, and get its corresponding operation.
    That is, if curr_job's first operation is not done yet, do first op;
    otherwise do second op. This value of which op to execute is stored in `job.next_op` attribute.

4. Calculate the best machine to schedule `curr_job`'s `curr_op`.
    The idea is to get its associated subset of machines, 
    and then order them by (1) their current finished times (for the sake of makespan), and then 
    (2) their `versatility`. Versatility is a static vector, `M.versatile[m]` specifies **the number of operations machine ${m}$ is capable of executing.** 
    To sum up, the best machine `curr_machine` is computed by: 
    ```curr_machine = min(avail_machines_idx, key = lambda m: (M.fintime[m], M.versatile[m], m))```
   Then put ```curr_job``` onto this machine.


5. To schedule or not to schedule, that is a question.
    
    **NEW FEATURE**
    Decide if this job is `PERMIT`: (1) if its current completion time plus current op's processing time is already tardy, associate it with a very big `LST`, set `PERMIT = False`. This seems to work on instance 3;
    (2) if its scheduling requires an idle time that is too large, postpone it temporarily, associate it with `LST`+0.5, set `PERMIT = False`.
    In both non-permitable cases, increment `failures` to record how many times the job is popped from ${Q}$ but fails to be scheduled. If > 2, we schedule it no matter waht. 
   
   If `PERMIT`, update values, especially update the completion_times, residual_times and **LST**,
   
   else(`not PERMIT`), push the operation back to ${Q}$ with the new `LST`. 
   
   
   For debugging purpose, I add time of idleness into M.schedule so that it is easier to read the schedule, calculate times and check feasibility. 

6. Check if all job operations are scheduled, if yes, stop the algorithm, if no, continue the iteration (go back to step 3).


#### Result:

1. `M.fintime` is the finishing time of machines, you can get the makespan by `max(M.fintime)`.
2. `M.schedule` is the schedule of machines, similar to gantt chart. 
3. `J.completion_times` is the completion times for all jobs. Comparing it with `J.due_dates` using
   ```tardies = list(np.where(J.completion_times > J.due_dates)[0])```
   gives you the tardy jobs (it's 0-indexed!!!). Turn it to 1-indexed by 
   ```[x+1 for x in tardies]```. 
   

In [11]:
# now put every operation into Q ( Q by operation )
from heapq import heappush, heappop, heapify
def make_Q(J):
    GAP = 0.1
    
    Job_keys = []
    for job_index in range(len(J.jobs)):
        op1_pt, op2_pt = J.jobs[job_index].stage_pt
        # 我覺得 due_dates 幾乎等於 total proc time的反而可以放後面(不太可能來得及，乾脆果斷放棄)
        LST = J.get_LST()
        if LST[job_index] <= GAP: # avoid division by zero error
            lst_ratios = [0, 0]
        else:
            lst_ratios = [op1_pt / J.LST[job_index], op2_pt / J.LST[job_index]]
        # preserve precedence:
        lst_ratios[0] = max(lst_ratios)
        # NOTE: min queue extracts by minimum value, so add a negative sign here
        # (lst_ratio, job_index, job_op)
        Job_keys.append((-lst_ratios[0], job_index, 0))
        Job_keys.append((-lst_ratios[1], job_index, 1))
    Q = Job_keys[:]
    heapify(Q) 
    return Q
Q = make_Q(J)

In [12]:
Q

[(-75.99999999999892, 12, 0),
 (-7.571428571428569, 17, 0),
 (-42.99999999999939, 12, 1),
 (-4.000000000000001, 4, 0),
 (-1.6551724137931032, 2, 0),
 (-19.2, 1, 0),
 (-4.45, 3, 0),
 (-3.6000000000000028, 16, 0),
 (-2.428571428571428, 17, 1),
 (-0.6250000000000001, 4, 1),
 (-0.847457627118644, 5, 0),
 (-0.847457627118644, 5, 1),
 (-2.042553191489361, 6, 0),
 (-0.3529411764705882, 14, 0),
 (-0.5670103092783505, 7, 0),
 (-0.09278350515463916, 7, 1),
 (-2.760000000000002, 16, 1),
 (-0.484375, 18, 0),
 (-0.4821428571428572, 9, 0),
 (-0.23214285714285718, 9, 1),
 (-0.3977272727272727, 10, 0),
 (-0.2954545454545454, 10, 1),
 (-0.651685393258427, 11, 0),
 (-0.651685393258427, 11, 1),
 (-0.4137931034482758, 2, 1),
 (-0.41666666666666674, 0, 0),
 (-0.9666666666666666, 13, 0),
 (-0.0, 13, 1),
 (-0.042553191489361694, 6, 1),
 (-0.044117647058823525, 14, 1),
 (-0.5486725663716814, 15, 0),
 (-0.053097345132743355, 15, 1),
 (-0.0, 1, 1),
 (-0.0, 3, 1),
 (-0.49253731343283585, 8, 0),
 (-0.029850746268

## Algorithm

In [13]:
# while not all operations in all jobs are scheduled

def heuristic(J, M): 
    # best_makepsan = sum(job_processing_time) for all jobs / |M|
    TOLRATIO = 0.3
    best_makespan = sum(job.stage_pt[0]+job.stage_pt[1] for job in J.jobs)/M.number
    tolerance = best_makespan * TOLRATIO  # tolerance for idle time
    # tolerance is a hyper-parameter; if idle > tolerance, do not schedule the curr op in the current epoch. 
    print(f'[INFO] {len(J.jobs)} jobs, {M.number} machines')
    print(f'[INFO] Tolerance: {tolerance:.2f}')
    # step 1. compute lst ratio for every operation 
    # step 2. make priority queue 
    Q = make_Q(J)
    fails = [0 for _ in range(len(J.jobs))]
    # step 6. check if all jobs are completed 
    epoch = 0
    while not np.all(J.is_completed):
        epoch += 1
        
        PERMIT = True
        # step 3. extract_min() to get the job with minimal LST and its other attributes
        _, curr_job_index, curr_op = heappop(Q)
        curr_job = J.jobs[curr_job_index]
        op_proc_time = curr_job.stage_pt[curr_op]
        job_name = (curr_job_index, curr_op)
         
        avail_machines_idx = [x-1 for x in curr_job.stage_mach[curr_op]]
        # if curr_job has no second operation 
        if op_proc_time <= 0: 
            J.assign(job_name = job_name, 
                    mach = None,
                    st = curr_job.end_time[curr_op-1]) 
            # note that it's only possible for second operation to have proc time = 0
            # so this doesn't trigger index error
            continue 
        # step 4. calculate the best machine
        curr_machine = min(avail_machines_idx, key = lambda x: (M.fintime[x], M.versatile[x], x))
        
        # ARE THERE REASONS TO POSTPONE THE CURR OP?
        if J.completion_times[curr_job_index] + op_proc_time > J.due_dates[curr_job_index] and fails[curr_job_index] < 2:
            print(f'[INFO] Job {curr_job_index+1} will be tardy even if scheduled, queue last.')
            curr_new_value = float('inf')
            PERMIT = False
        
        # ARE THERE REASONS TO POSTPONE THE CURR OP?
        elif M.fintime[curr_machine] < J.completion_times[curr_job_index]:
            idle = J.completion_times[curr_job_index] - M.fintime[curr_machine]
            if idle > tolerance and curr_op == 1 and fails[curr_job_index] < 2:
                print(f'[INFO] Job {curr_job_index+1} op {curr_op} has idle {idle:.2f}, postpone it.')
                PERMIT = False
                if Q:
                    curr_new_value = Q[0][0] + 1
                else:
                    curr_new_value = 0 # the last one 
            else:
                M.add_idle( 
                mach = curr_machine, 
                idle_time = idle)
         
        if PERMIT:
            J.assign(job_name = job_name, 
                mach = curr_machine, 
                 st = M.fintime[curr_machine]
                ) 
            M._schedule(job_name = job_name, 
               mach = curr_machine, 
                proc_time = op_proc_time,
               st = M.fintime[curr_machine])
            curr_new_value = J.get_LST()[curr_job_index]
        else: 
            fails[curr_job_index] += 1
        # print(f'{epoch} Fails Count:', fails)
        # update the LST value and push it back to Q if the job has its second operation that hasn't been done
        if not PERMIT:
            heappush(Q, (curr_new_value, curr_job_index, curr_op))
            # it maintains the heap invariant, no need to heapify

In [ ]:
'''
for i in range(len(queue.queue)):
    temp = queue.get()
    queue.put(temp)
    print temp
'''

In [14]:
heuristic(J, M)
J.is_completed

[INFO] 20 jobs, 9 machines
[INFO] Tolerance: 5.55
[INFO] Job 13 op 1 has idle 7.60, postpone it.
[INFO] Job 13 op 1 has idle 7.60, postpone it.
[INFO] Job 17 op 1 has idle 9.00, postpone it.
[INFO] Job 6 will be tardy even if scheduled, queue last.
[INFO] Job 3 will be tardy even if scheduled, queue last.
[INFO] Job 11 will be tardy even if scheduled, queue last.
[INFO] Job 19 will be tardy even if scheduled, queue last.
[INFO] Job 15 will be tardy even if scheduled, queue last.
[INFO] Job 20 will be tardy even if scheduled, queue last.
[INFO] Job 3 will be tardy even if scheduled, queue last.
[INFO] Job 6 will be tardy even if scheduled, queue last.
[INFO] Job 11 will be tardy even if scheduled, queue last.
[INFO] Job 15 will be tardy even if scheduled, queue last.
[INFO] Job 19 will be tardy even if scheduled, queue last.
[INFO] Job 20 will be tardy even if scheduled, queue last.


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [15]:
M.span

[[6.4, 3.5, 6.6, 0.6, 1.2, 0.3999999999999986, 2.6],
 [9.6, 6.2, 0.6999999999999993, 0.4, 6.399999999999999, 6.6],
 [7.6, 4.3, 4.9, 0.5],
 [5.3, 1.7, 2.0, 6.9, 4.8],
 [9.6, 1.0, 5.4, 0.9, 5.0],
 [7.6, 8.7, 0.2, 6.8],
 [9.0, 0.40000000000000036, 5.8, 3.5],
 [5.3, 4.8, 3.1, 2.8000000000000007, 2.6, 2.099999999999998, 0.6],
 [8.9, 0.5, 5.5, 1.8999999999999968, 5.0]]

In [16]:
J.completion_times

array([21.8,  9.6, 18.3,  8.9, 10.6, 21.9, 16.5, 16.9, 16.9, 18.6, 21.3,
       15.2, 11.9, 16.3, 21.3, 17.1, 15.9,  7. , 17.3, 29.9])

In [17]:
np.where(J.completion_times > J.due_dates)

(array([ 2,  4,  5,  6,  7, 10, 14, 18, 19]),)

In [18]:
Tardy_jobs = list(np.where(J.completion_times > J.due_dates)[0])
Makespan = max(M.fintime)
print(f'Instance {i+1}:')
print('First objective (# tardy):', len(Tardy_jobs), Tardy_jobs)
print('Second objective (makespan):', Makespan)

Instance 5:
First objective (# tardy): 9 [2, 4, 5, 6, 7, 10, 14, 18, 19]
Second objective (makespan): 29.9


In [19]:
M.fintime

[21.3,
 29.9,
 17.299999999999997,
 20.7,
 21.9,
 23.299999999999997,
 18.7,
 21.3,
 21.799999999999997]

In [20]:
# 類似Gantt Chart
# () is 1-indexed job name, the attached value is its associated proc time
# idle is the gap appears to preserve precedence between the operations within the same job 
print(*M.schedule, sep = '\n')

['(5, 1) 6.40', '(6, 1) 3.50', '(9, 1) 6.60', '(16, 2) 0.60', '(3, 2) 1.20', 'idle 0.40', '(11, 2) 2.60']
['(2, 1) 9.60', '(16, 1) 6.20', 'idle 0.70', '(9, 2) 0.40', 'idle 6.40', '(20, 2) 6.60']
['idle 7.60', '(13, 2) 4.30', '(1, 1) 4.90', '(19, 2) 0.50']
['idle 5.30', '(18, 2) 1.70', 'idle 2.00', '(17, 2) 6.90', '(15, 1) 4.80']
['(7, 1) 9.60', '(5, 2) 1.00', '(10, 1) 5.40', '(8, 2) 0.90', '(6, 2) 5.00']
['(13, 1) 7.60', '(14, 1) 8.70', '(7, 2) 0.20', '(20, 1) 6.80']
['(17, 1) 9.00', 'idle 0.40', '(12, 2) 5.80', '(11, 1) 3.50']
['(18, 1) 5.30', '(3, 1) 4.80', '(19, 1) 3.10', 'idle 2.80', '(10, 2) 2.60', 'idle 2.10', '(15, 2) 0.60']
['(4, 1) 8.90', '(12, 1) 0.50', '(8, 1) 5.50', 'idle 1.90', '(1, 2) 5.00']


## Testing all instances

In [21]:
Results = []
for inst in range(5):
    data = instances[inst]
    M = Machines(data)
    J = Jobs(len(data))
    J.add_jobs(data)
    heuristic(J = J, M = M)
    print(f'** Summary \ninstance {inst+1}:')
    Tardy_jobs = list(np.where(J.completion_times > J.due_dates)[0])
    Tardy_jobs = [x+1 for x in Tardy_jobs]
    Makespan = max(M.fintime)
    print('First objective (# tardy):', len(Tardy_jobs), Tardy_jobs)
    print('Second objective (makespan):', Makespan)
    print('==================================')
    Results.append({'J':J, 'M':M})

[INFO] 12 jobs, 5 machines
[INFO] Tolerance: 1.81
[INFO] Job 1 op 1 has idle 2.70, postpone it.
[INFO] Job 4 will be tardy even if scheduled, queue last.
[INFO] Job 10 op 1 has idle 3.10, postpone it.
[INFO] Job 10 op 1 has idle 3.10, postpone it.
[INFO] Job 4 will be tardy even if scheduled, queue last.
** Summary 
instance 1:
First objective (# tardy): 1 [4]
Second objective (makespan): 7.9
[INFO] 11 jobs, 5 machines
[INFO] Tolerance: 2.21
[INFO] Job 7 will be tardy even if scheduled, queue last.
[INFO] Job 7 will be tardy even if scheduled, queue last.
** Summary 
instance 2:
First objective (# tardy): 1 [7]
Second objective (makespan): 9.3
[INFO] 10 jobs, 5 machines
[INFO] Tolerance: 2.33
[INFO] Job 1 op 1 has idle 3.00, postpone it.
[INFO] Job 2 will be tardy even if scheduled, queue last.
[INFO] Job 3 will be tardy even if scheduled, queue last.
[INFO] Job 2 will be tardy even if scheduled, queue last.
[INFO] Job 3 will be tardy even if scheduled, queue last.
** Summary 
instance

In [22]:
see = 1
print(*Results[see]['M'].schedule, sep = '\n')

['(2, 1) 1.60', '(5, 1) 0.80', '(1, 1) 2.70', 'idle 1.30', '(11, 2) 1.50']
['idle 1.60', '(2, 2) 2.30', '(11, 1) 2.50', '(1, 2) 1.50']
['(3, 1) 1.00', '(6, 1) 2.70', '(7, 1) 1.40', '(8, 1) 2.20', 'idle 0.20', '(9, 2) 1.80']
['idle 1.00', '(3, 2) 2.70', '(4, 2) 0.80', '(10, 1) 2.20', '(9, 1) 0.80', '(7, 2) 1.50']
['(4, 1) 2.80', '(5, 2) 1.90', 'idle 2.00', '(10, 2) 2.20']


In [23]:
print(*Results[see]['J'].due_dates, sep = ', ')
print('====')
print(*Results[see]['J'].completion_times, sep = ', ')

10, 5, 5, 5, 5, 5, 5, 10, 10, 10, 10
====
7.9, 3.9, 3.7, 4.5, 4.699999999999999, 3.7, 9.0, 7.3, 9.3, 8.9, 7.9


In [66]:
R[see][1]

[[5.1, 7.9],
 [1.6, 3.9],
 [1.0, 3.7],
 [2.8, 4.5],
 [2.4, 4.7],
 [3.7, 3.7],
 [5.1, 9.0],
 [7.3, 7.3],
 [7.5, 9.3],
 [6.7, 8.9],
 [6.4, 7.9]]

In [67]:
instances[1]

,Job ID,Stage-1 Processing Time,Stage-2 Processing Time,Stage-1 Machines,Stage-2 Machines,Due Time
0,1,2.7,1.5,"1,2,3,4,5","2,3,4,5",10
1,2,1.6,2.3,"1,2,3,4,5","2,3,4,5",5
2,3,1.0,2.7,"1,2,3,4,5","2,3,4,5",5
3,4,2.8,0.8,"2,3,4,5","2,3,4,5",5
4,5,0.8,1.9,"1,2,3,4,5","2,3,4,5",5
5,6,2.7,0.0,"1,2,3,4,5",NaN,5
6,7,1.4,1.5,"2,3,4,5","2,3,4,5",5
7,8,2.2,0.0,"2,3,4,5",NaN,10
8,9,0.8,1.8,"1,2,3,4,5","2,3,4,5",10
9,10,2.2,2.2,"2,3,4,5","2,3,4,5",10


## Output

1. Write your heuristic algorithm here.
2. We would call this function in CA2_grading_program.py to evaluate your algorithm.
3. Please do not change the function name and the file name.
4. The parameter is the file path of a data file, whose format is specified in the document. 
5. You need to return your schedule in two lists "machine" and "completion_time".
    (a) machine[j][0] is the machine ID of the machine to process the first stage of job j + 1, and 
machine[j][1] is the machine to process the second stage of job j + 1.
    (b) completion_time[j][0] is the completion time of the first stage of job j + 1, and 
completion_time[j][1] is the completion time of the second stage of job j + 1. 
    
    
    Note 1. If you have n jobs, both the two lists are n by 2 (n rows, 2 columns). 
    Note 2. In the list "machine", you should record the IDs of machines 
            (i.e., to let machine 1 process the first stage of job 1, 
            you should have machine[0][0] == 1 rather than machine[0][0] == 0).
6. You only need to submit this algorithm_module.py.

**How about those with only one stage???**

In [61]:
def make_result(J):
    '''pass in the resulted J'''
    n = len(J.jobs)
    machine, completion_time = [], []
    for i in range(n):
        if  J.jobs[i].assign_mach[1] is None:
            op2_mach_id = None
        else: op2_mach_id = J.jobs[i].assign_mach[1]+1
        op1_mach_id = J.jobs[i].assign_mach[0]+1
        op1_c_time = round(J.jobs[i].end_time[0], 3)
        op2_c_time = round(J.jobs[i].end_time[1], 3)
        machine.append([op1_mach_id, op2_mach_id])
        completion_time.append([op1_c_time, op2_c_time])
    assert len(machine) == len(completion_time) == n
    return machine, completion_time

In [62]:
R = make_result(Results[0]['J']) # TA要的格式 

In [65]:
import joblib 
filepath = './OR_Case2_LSTratio_sol.pkl'
R = [make_result(Results[i]['J']) for i in range(5)]
joblib.dump(R, filepath)

['./OR_Case2_LSTratio_sol.pkl']